In [1]:
import os
import json

In [2]:
json_wd = os.path.join(os.getcwd(),'json')
print (json_wd)

C:\Users\ephra\Documents\GitHub\PIDA_Project\json


In [3]:
from os import listdir
from os.path import isfile, join

jsonfiles = [f for f in listdir(json_wd) if isfile(join(json_wd, f))]

In [4]:
print(len(jsonfiles))

2


In [5]:
#input all json files and load into list
all_files=[]
for filename in jsonfiles:
    file=open(os.path.join(json_wd, filename))
    data=json.load(file)
    all_files.append(data)
    file.close()
    #data=json.loads(filename)

In [6]:
#way to access all needed values
#print(all_files[0])
first_key = list(all_files[0].keys())[0] #this is the name of the dataset
print (first_key, " is the name of the dataset") 
print (all_files[0][first_key], "all columns and tags on dataset") 
for i in all_files[0][first_key].keys():
    print (all_files[0][first_key][i])

Bob  is the name of the dataset
{'data': ['along', 'bob', 'another'], 'ele': ['another', 'again']} all columns and tags on dataset
['along', 'bob', 'another']
['another', 'again']


In [7]:
all_links=[]
for index, file in enumerate(all_files):
    #print (list(file.keys())[0], "is the name of the dataset")
    name=list(file.keys())[0]
    #print (list(file[name].keys()), "are the column names")
    for col in file[name].keys():
        #print ("Under column ", col, "are tags ", file[name][col])
        for tag in file[name][col]:
            #print ("Tag: ", tag)
            for secondindex in range (index+1, len(all_files)):
                secondname=list(all_files[secondindex].keys())[0]
                for secondcol in all_files[secondindex][secondname].keys():
                    #print ("Under second column ", secondcol, "are tags ", all_files[secondindex][secondname][secondcol])
                    for secondtag in all_files[secondindex][secondname][secondcol]:
                        #print ("Tag: ", secondtag)
                        #do all checking for common tags here
                        if tag==secondtag:
                            print (name, " and ", secondname, " are linked at column ", col, " and column ", secondcol, "through tag", tag)
                            all_links.append([name, col, tag , secondname, secondcol, secondtag])
                        #do all checking for common tags here
print (len(all_links))

Bob  and  311  are linked at column  data  and column  test1 through tag another
Bob  and  311  are linked at column  ele  and column  test1 through tag another
Bob  and  311  are linked at column  ele  and column  test1 through tag again
3
